[Home](../../index.ipynb) / [Web-Server](../index.ipynb) / Simple car
***

# Simple car

Nach dem Start der Programme das WLAN des Controllers auswählen und die auf der Console angezeigte IP-Adresse ansurfen (meist 192.168.4.1).

In [97]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB10 --baud=115200 
Ready.


In [98]:
from httpServer import RequestHandler, FileRequestHandler, ContentType, SimpleAccessPointHttpServer
from motor import Motor
import display

class SpeedRequestHandler( RequestHandler ) :
    def __init__( self ) :
        super().__init__( strContentType = ContentType.PLAIN )
        self._fCurV    = 0
        self._fCurGear = 0
        self._motorL = Motor( 12, 13,
            iFrequency     = 300,
            iDutyMinLeft   = 220,
            iDutyMaxLeft   = 1023,
            iDutyMinRight  = 220,
            iDutyMaxRight  = 1023,
            fSpeedMaxLeft  = -1.0,
            fSpeedMaxRight =  1.0 )

        self._motorR = Motor( 18, 22,
            iFrequency     = 300,
            iDutyMinLeft   = 220,
            iDutyMaxLeft   = 1023,
            iDutyMinRight  = 220,
            iDutyMaxRight  = 1023,
            fSpeedMaxLeft  = -1.0,
            fSpeedMaxRight =  1.0 )
        
        
    def dispatch( self, socketClient, strPath = None, match = None ) :
        # Here You have velocity in[-1;1] and gear in [-1;1]
        
        fGear = float(match.group( 1 )) # x
        fV    = float(match.group( 2 )) # y
        
        if fV != self._fCurV or fGear != self._fCurGear :
            self._motorL.setSpeed( fV + fGear )
            self._motorR.setSpeed( fV - fGear )
            
            self._fCurV    = fV
            self._fCurGear = fGear
            
            print( "Set: Velocity = {}, Gear = {}".format( fV, fGear ), end="" )
            
            return True
        
        else:
            print( "NOT set: Velocity = {}, Gear = {}".format( fV, fGear ), end="" )
        
        self.sendHeader( socketClient )
        socketClient.sendall( str(fV) + "_" + str(fGear) )


# httpServer = httpServer.SimpleAccessPointHttpServer()
httpServer = SimpleAccessPointHttpServer()


httpServer.addRequestHandler( "/joystick/(.*)_(.*)", SpeedRequestHandler() )
httpServer.addRequestHandler( "/default.css", FileRequestHandler( "/resources/default.css", strContentType = ContentType.CSS        ) )
httpServer.addRequestHandler( "/default.js",  FileRequestHandler( "/resources/default.js",  strContentType = ContentType.JAVASCRIPT ) )
httpServer.addRequestHandler( "/",            FileRequestHandler( "/resources/index.html" ) )


httpServer.start( ssid = "ESP32 Simple car", password = "123123123123" )

print( httpServer )

display = display.Display()

display.clear()
display.text( httpServer.ipAddress, 0, 10 )
display.show()


# >>> disable the brownout-detector
from machine import mem32
from micropython import const

DR_REG_RTCCNTL_BASE    = const(0x3ff48000)
RTC_CNTL_BROWN_OUT_REG = const(DR_REG_RTCCNTL_BASE + 0xd4)
mem32[RTC_CNTL_BROWN_OUT_REG] = 0
# <<< disable the brownout-detector


#######################################
# Loop

try :
    while True:
        print( "\nWaiting for client...", end="" )
        httpServer.accept()
        
except KeyboardInterrupt:
    httpServer.stop()
    print( "\nServer stopped." )
    display.text( "Server stopped.", 0, 30 )
    display.show()

192.168.4.1
Name: SimpleAccessPointHttpServer, IP address: 192.168.4.1, Subnet mask: 255.255.255.0, Gateway: 192.168.4.1, DNS server: 0.0.0.0, MAC address: 80:7d:3a:da:d3:78

Waiting for client......
Waiting for client...
Waiting for client...
Waiting for client...Set: Velocity = -0.121, Gear = 0.008
Waiting for client...Set: Velocity = -0.152, Gear = 0.008
Waiting for client...Set: Velocity = -0.174, Gear = 0.008
Waiting for client...Set: Velocity = -0.182, Gear = 0.015
Waiting for client....Set: Velocity = 0.0, Gear = 0.0
Waiting for client...Set: Velocity = -0.144, Gear = 0.0
Waiting for client...NOT set: Velocity = -0.144, Gear = 0.0
Waiting for client...Set: Velocity = -0.212, Gear = 0.008
Waiting for client...Set: Velocity = -0.22, Gear = 0.008
Waiting for client...Set: Velocity = -0.227, Gear = 0.008
Waiting for client...Set: Velocity = -0.242, Gear = 0.008
Waiting for client...Set: Velocity = -0.25, Gear = 0.008
Waiting for client...Set: Velocity = -0.258, Gear = 0.008
Waiting 

In [ ]:
%serialconnect
%sendtofile --source "motor.py" 